In [1]:
import numpy as np
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from skimage.io import imshow
from keras import backend as K
from keras.constraints import Constraint
from keras.datasets import fashion_mnist
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
def inference_via_confidence(confidence_mtx1, confidence_mtx2, label_vec1, label_vec2):

    #----------------First step: obtain confidence lists for both training dataset and test dataset--------------
    confidence1 = []
    confidence2 = []
    acc1 = 0
    acc2 = 0
    for num in range(confidence_mtx1.shape[0]):
        confidence1.append(confidence_mtx1[num,label_vec1[num]])
        if np.argmax(confidence_mtx1[num,:]) == np.argmax(label_vec1[num]):
            acc1 += 1

    for num in range(confidence_mtx2.shape[0]):
        confidence2.append(confidence_mtx2[num,label_vec2[num]])
        if np.argmax(confidence_mtx2[num,:]) == np.argmax(label_vec2[num]):
            acc2 += 1
    confidence1 = np.array(confidence1)
    confidence2 = np.array(confidence2)

    print('model accuracy for training and test-', (acc1/confidence_mtx1.shape[0], acc2/confidence_mtx2.shape[0]) )


    #sort_confidence = np.sort(confidence1)
    sort_confidence = np.sort(np.concatenate((confidence1, confidence2)))
    max_accuracy = 0.5
    best_precision = 0.5
    best_recall = 0.5
    for num in range(len(sort_confidence)):
        delta = sort_confidence[num]
        ratio1 = np.sum(confidence1>=delta)/confidence_mtx1.shape[0]
        ratio2 = np.sum(confidence2>=delta)/confidence_mtx2.shape[0]
        accuracy_now = 0.5*(ratio1+1-ratio2)
        if accuracy_now > max_accuracy:
            max_accuracy = accuracy_now
            best_precision = ratio1/(ratio1+ratio2)
            best_recall = ratio1
    print('maximum inference accuracy is:', max_accuracy)
    print('precision is:', best_precision)
    print('recall is:', best_recall)
    return max_accuracy


In [0]:
class Sparse(Constraint):
    '''
    We will use one variable: Mask
    After we train our model dense model,
    we will save the weights and analyze them.
    We will create a mask where 1 means the
    number is far away enough from 0 and 0
    if it is to close to 0. We will multiply
    the weights by 0(making them 0) if they
    are supposed to be masked.
    '''

    def __init__(self, mask):
        self.mask = K.cast_to_floatx(mask)

    def __call__(self,x):
        return self.mask * x

    def get_config(self):
        return {'mask': self.mask}


def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for weights in weights_list:
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparsity))*1.)
    return masks

# **Dense Training**

In [5]:
epochs=50

model = Sequential([
    Conv2D(32,3,input_shape=(28,28,1)),
    Activation('relu'),
    MaxPooling2D(2,2),
    Conv2D(64,3),
    Activation('relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(250),
    Activation('relu'),
    Dense(10),
    Activation('softmax')
])

adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


model.fit(x_train, y_train,
          batch_size=32,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))









Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 11, 11, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)       

In [6]:
output_train=model.predict(x_train)
output_test=model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)

model accuracy for training and test- (0.9934833333333334, 0.9053)
maximum inference accuracy is: 0.5530999999999999
precision is: 0.5146507008056506
recall is: 1.8653
Maximum Accuracy: 0.5530999999999999


#  **Sparse Training**

In [23]:
masks = create_sparsity_masks(model,40)#Closest 30% to 0

sparse_model =  Sequential([
    Conv2D(32,3,input_shape=(28,28,1), kernel_constraint=Sparse(masks[0])),
    Activation('relu'),
    MaxPooling2D(2,2),
    Conv2D(64,3, kernel_constraint=Sparse(masks[1])),
    Activation('relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(250, kernel_constraint=Sparse(masks[2])),
    Activation('relu'),
    Dense(10, kernel_constraint=Sparse(masks[3])),
    Activation('softmax')
])

adam = Adam()
sparse_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
sparse_model.summary()
#Get weights from densely trained model
sparse_model.set_weights(model.get_weights())


# Train Sparse Model
sparse_model.fit(x_train,y_train,
          batch_size=32,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
activation_37 (Activation)   (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
activation_38 (Activation)   (None, 11, 11, 64)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1600)            

In [24]:
output_train=sparse_model.predict(x_train)
output_test=sparse_model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)

model accuracy for training and test- (0.9992666666666666, 0.9117)
maximum inference accuracy is: 0.5735833333333332
precision is: 0.5196307725142506
recall is: 1.9477666666666666
Maximum Accuracy: 0.5735833333333332
